In [ ]:
grid = open('input.txt').read().splitlines()

rows = len(grid)
cols = len(grid[0])

time_grid = []

for i in range(rows):
    line = []
    for j in range(cols):
        line.append((grid[i][j], (0 if grid[i][j] != '#' else -1)))
        if grid[i][j] == 'S':
            start = (i, j)
        if grid[i][j] == 'E':
            end = (i, j)
    time_grid.append(line)

grid, rows, cols, start, end, time_grid

In [ ]:
import heapq

def in_grid(pos):
    return 0 <= pos[0] < rows and 0 <= pos[1] < cols

def is_wall(pos, grid):
    return grid[pos[0]][pos[1]] == '#'

def is_end(pos):
    return pos == end

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(start, end, grid):
    global rows, cols
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, end)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == end:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path
        
        for dir in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor = (current[0] + dir[0], current[1] + dir[1])
            if in_grid(neighbor) and not is_wall(neighbor, grid):
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, end)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None

path = a_star(start, end, grid)
print(len(path) - 1)
print(path)

for idx, pos in enumerate(path):
    time_grid[pos[0]][pos[1]] = (time_grid[pos[0]][pos[1]][0], idx)

time_grid


In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

def time_to_color(time, max_time):
    if time == -1:
        return (0, 0, 0)  # Black
    elif time == 0:
        return (255, 0, 0)  # Red
    elif time == max_time:
        return (0, 255, 0)  # Green
    else:
        # Calculate the gradient from red to green
        red = int(255 * (1 - time / max_time))
        green = int(255 * (time / max_time))
        return (red, green, 0)
    
def draw_time_grid(time_grid):
    rows = len(time_grid)
    cols = len(time_grid[0])
    all_times = [time for row in time_grid for _, time in row if time != -1]
    max_time = max(all_times)

    # Create a new image with a gray background
    image = Image.new('RGB', (cols * 20, rows * 20))
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for y in range(rows):
        for x in range(cols):
            char, time = time_grid[y][x]
            color = time_to_color(time, max_time)
            # Draw the background rectangle
            draw.rectangle([x * 20, y * 20, x * 20 + 20, y * 20 + 20], fill=color)
            # Draw the character in white
            draw.text((x * 20 + 5, y * 20 + 5), char, fill=(200,200,200), font=font)



    return image

image = draw_time_grid(time_grid)
image = image.resize((cols * 50, rows * 50), Image.NEAREST)
display(image)

In [ ]:

cheats = [
    (0,2),
    (1,1),
    (2,0),
    (1,-1),
    (0,-2),
    (-1,-1),
    (-2,0),
    (-1,1)
]

time_saves = {}

for pos in path:
    for cheat in cheats:
        neighbor = (pos[0] + cheat[0], pos[1] + cheat[1])
        if in_grid(neighbor) and time_grid[neighbor[0]][neighbor[1]][1] != -1 and time_grid[neighbor[0]][neighbor[1]][1] > time_grid[pos[0]][pos[1]][1] + 2:
            time_saved = time_grid[neighbor[0]][neighbor[1]][1] - time_grid[pos[0]][pos[1]][1] - 2
            time_saves[time_saved] = time_saves.get(time_saved, 0) + 1
            # print(f"Found a time save of {time_saved} from {pos} to {neighbor}")
            

time_saves = dict(sorted(time_saves.items()))

ans = 0
for time_saved, count in time_saves.items():
    print(f"{count} time saves of {time_saved} steps")
    if time_saved >= 100:
        ans += count
    
print(f"{ans} time saves of 100 steps or more")

In [ ]:

def draw_cheats(cheats):
    rows = cheat_length * 2 + 1
    cols = cheat_length * 2 + 1

    # Create a new image with a gray background
    image = Image.new('RGB', (cols * 20, rows * 20))
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for y in range(rows):
        for x in range(cols):
            color = (255, 255, 255)
            # Draw the background rectangle
            draw.rectangle([x * 20, y * 20, x * 20 + 20, y * 20 + 20], fill=(color if (x - rows//2, y - cols//2) in cheats else (0, 0, 0)))

    return image

cheat_length = 20

cheats = []
for i in range(-cheat_length, cheat_length + 1):
    for j in range(-cheat_length,cheat_length + 1):
        if i == 0 and j == 0:
            continue
        if abs(i) + abs(j) > cheat_length:
            continue
        
        cheats.append((i, j))

image = draw_cheats(cheats)
image = image.resize((cheat_length * 20, cheat_length * 20), Image.NEAREST)
display(image)
        

time_saves = {}

for pos in path:
    for cheat in cheats:
        neighbor = (pos[0] + cheat[0], pos[1] + cheat[1])
        cheat_time = abs(cheat[0]) + abs(cheat[1])
        if in_grid(neighbor) and time_grid[neighbor[0]][neighbor[1]][1] != -1 and time_grid[neighbor[0]][neighbor[1]][1] > time_grid[pos[0]][pos[1]][1] + cheat_time:
            time_saved = time_grid[neighbor[0]][neighbor[1]][1] - time_grid[pos[0]][pos[1]][1] - cheat_time
            time_saves[time_saved] = time_saves.get(time_saved, 0) + 1
            # print(f"Found a time save of {time_saved} from {pos} to {neighbor}")
            

time_saves = dict(sorted(time_saves.items()))

ans = 0
for time_saved, count in time_saves.items():
    print(f"{count} time saves of {time_saved} steps")
    if time_saved >= 100:
        ans += count
    
print(f"{ans} time saves of 100 steps or more")
